連線database

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=1000)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# 可自由調整select的區間
# today_str = '20240813'
today_str = '20241231'
one_year_ago_str = '20210101'

# SQL 查詢 加權指數
query_daily_tx = f"""
SELECT *
FROM daily_tx
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 加權指數
query_daily_updown = f"""
SELECT *
FROM daily_updown
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 加權指數
query_tif_investors = f"""
SELECT *
FROM daily_tif_investors
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 加權指數
query_pc_ratio = f"""
SELECT *
FROM daily_pc_ratio
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
tif_investors_df_raw = pd.read_sql(query_tif_investors, engine)
# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
updown_df_raw = pd.read_sql(query_daily_updown, engine)
# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
tx_df_raw = pd.read_sql(query_daily_tx, engine)

pc_ratio_raw = pd.read_sql(query_pc_ratio, engine)

stock_df_raw = pd.read_sql(query_stock, engine)
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])

In [2]:
# 每天成交金額
stock_date_list = stock_df_raw[(stock_df_raw['Date']>='2024-12-10' ) & (stock_df_raw['Date']<='2024-12-17' ) & (stock_df_raw['證券代號'] == '2330' ) ]['Date']
collected_data = pd.DataFrame(columns=['證券代號','證券名稱', '收盤價', 'Date'])

for day in stock_date_list:
        stock_df_today = stock_df_raw[(stock_df_raw['Date'] == day)].sort_values(by='成交金額',ascending=False).head(30)

In [23]:
import pandas as pd

# 篩選指定日期範圍的資料
stock_date_list = stock_df_raw[
    (stock_df_raw['Date'] >= '2024-11-01') & 
    (stock_df_raw['Date'] <= '2024-12-20') & 
    (stock_df_raw['證券代號'] == '2330')
]['Date'].unique()

# 初始化 DataFrame 紀錄每日變動
change_log = pd.DataFrame(columns=['Date', '新增股票', '被踢出股票', '濾網股票'])

# 初始化前一天排行榜的股票列表
previous_top_stocks = set()

for idx, day in enumerate(stock_date_list):
    # 當日排行榜股票（Top 30，依成交金額排序）
    stock_df_today = stock_df_raw[stock_df_raw['Date'] == day].sort_values(by='成交金額', ascending=False).head(20)
    # stock_df_today_filter = stock_df_today[(stock_df_today['20RS_rank'] > 90) ]
    current_top_stocks = set(stock_df_today['證券名稱'].values)

    # 新增濾網
    stock_df_today_filter = stock_df_today[(stock_df_today['20RS_rank'] > 90) & (stock_df_today['收盤價'] > stock_df_today['開盤價'])  ]

    if idx > 0:  # 從第二天開始記錄變化
        # 計算進入和被踢出的股票
        new_entries = current_top_stocks - previous_top_stocks  # 當天新增進入的股票
        removed_entries = previous_top_stocks - current_top_stocks  # 當天被踢出的股票

        # 新增條件的股票
        filter_stock = set(stock_df_today_filter['證券名稱'].values)


        # 將變化紀錄到 DataFrame
        change_log = pd.concat([
            change_log,
            pd.DataFrame({
                'Date': [day],
                '新增股票': [', '.join(new_entries)],
                '被踢出股票': [', '.join(removed_entries)],
                '濾網股票' : [', '.join(new_entries & filter_stock)],
            })
        ], ignore_index=True)
    
    # 更新前一天的排行榜
    previous_top_stocks = current_top_stocks

# 查看結果
change_log

C:\Users\USER\AppData\Local\Temp\ipykernel_13268\4223425734.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  change_log = pd.concat([


,Date,新增股票,被踢出股票,濾網股票
0,2024-11-04,"上詮, 緯穎, 奇鋐, 萬潤","日月光投控, 聯電, 創意, 晟銘電",
1,2024-11-05,"創威, 信驊, 華城, 晟銘電, 聯電, 英業達","緯穎, 欣興, 上詮, 萬潤, 奇鋐, 台達電",英業達
2,2024-11-06,"菱光, 金寶, 奇鋐, 萬潤, 京元電子","陽明, 創威, 信驊, 華城, 光聖","金寶, 京元電子"
3,2024-11-07,"欣興, 元太, 川湖, 華城, 鈊象","萬海, 菱光, 金寶, 萬潤, 雙鴻","欣興, 川湖"
4,2024-11-08,"萬海, 陽明, 神達, 華星光, 智邦, 正崴, 雙鴻, 精材","元太, 技嘉, 華城, 晟銘電, 京元電子, 鈊象, 聯電, 英業達","萬海, 陽明, 神達, 華星光, 正崴"
5,2024-11-11,"緯穎, 華碩, 力旺, 華城, 貿聯-KY, 晟銘電, 京元電子, 英業達","欣興, 神達, 智邦, 奇鋐, 正崴, 川湖, 雙鴻, 精材","貿聯-KY, 華城, 晟銘電, 英業達"
6,2024-11-12,"神達, 中租-KY, 順達, 元大台灣50, 台達電","華碩, 華星光, 貿聯-KY, 長榮, 京元電子","神達, 順達"
7,2024-11-13,"日月光投控, 仁寶, 智邦, 弘塑, 長榮, 旺矽","萬海, 陽明, 英業達, 力旺, 元大台灣50, 台達電","智邦, 弘塑"
8,2024-11-14,"華碩, 新盛力, 奇鋐, 長榮航, 保瑞, 雙鴻, 聯電","日月光投控, 仁寶, 智邦, 弘塑, 華城, 中租-KY, 旺矽","雙鴻, 華碩, 長榮航"
9,2024-11-15,"陽明, 金寶, 技嘉, 貿聯-KY, 鈊象","新盛力, 保瑞, 順達, 聯電, 世芯-KY","金寶, 貿聯-KY, 陽明"


In [17]:
stock_df_today

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,60MA,60RS,20RS_sd,20RS_rank,60RS_sd,60RS_rank,240MA,240RS,240RS_sd,240RS_rank
1717215,2024-12-16,2330,台積電,43679921,53568,47253709874,1085.0,1085.0,1075.0,1085.0,...,1033.9167,104.9408,1.033066,92.864750,1.041800,89.616613,869.0833,124.8442,1.146996,93.450479
1717747,2024-12-16,3661,世芯-KY,6833561,22352,20522423840,2985.0,3065.0,2920.0,3020.0,...,2213.5833,136.4304,1.223217,99.946752,1.354413,99.574015,2846.2083,106.1061,0.974841,80.191693
1717208,2024-12-16,2317,鴻海,75228642,96744,13834643194,188.0,189.5,181.5,182.0,...,200.7250,90.6713,0.917298,19.116081,0.900139,29.872204,166.8079,109.1075,1.002416,83.439830
1717657,2024-12-16,3443,創意,8359102,16157,11728386075,1355.0,1455.0,1355.0,1365.0,...,1203.3833,113.4302,1.094038,98.189563,1.126078,96.911608,1357.5333,100.5500,0.923795,70.713525
1717241,2024-12-16,2368,金像電,47110820,34499,11406498431,236.0,250.5,233.5,240.5,...,194.1083,123.8999,1.205108,99.787007,1.230016,98.828541,210.4625,114.2721,1.049866,88.125666
1717300,2024-12-16,2454,聯發科,7521031,18946,10583170115,1435.0,1440.0,1385.0,1385.0,...,1275.5000,108.5849,1.048279,94.781683,1.077977,93.982961,1180.1292,117.3600,1.078235,90.042599
1717489,2024-12-16,3019,亞光,73395836,48564,9750109145,130.0,136.0,129.5,131.5,...,110.8583,118.6199,1.147293,99.361022,1.177599,97.816826,85.3092,154.1452,1.416197,98.083067
1717660,2024-12-16,3450,聯鈞,38077005,53348,9484509897,260.0,267.5,232.0,232.0,...,226.3333,102.5037,0.954489,43.769968,1.017605,85.037274,126.9208,182.7911,1.679378,99.148030
1717251,2024-12-16,2382,廣達,30402631,41273,8472953608,287.0,287.5,274.0,275.0,...,292.3083,94.0787,0.936220,29.925453,0.933966,45.473908,271.6813,101.2216,0.929965,72.044728
1717771,2024-12-16,3706,神達,109475574,50773,8354419349,79.4,79.6,73.8,75.8,...,60.2900,125.7257,0.913972,17.039404,1.248142,99.201278,49.0044,154.6801,1.421111,98.136315
